In [ ]:
import time
import random
import requests
import re
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import string
import pickle
import os

from bs4 import BeautifulSoup
import html.parser
import urllib.parse
from html.parser import HTMLParser
from flask import Flask, request, jsonify, render_template

In [ ]:
url = 'https://raw.githubusercontent.com/YaronRic/final_task/main/output_all_students_Train_v8.csv'
df = pd.read_csv(url)
df.head(1)

,City,type,room_number,Area,Street,number_in_street,city_area,price,num_of_images,floor_out_of,...,condition,hasAirCondition,hasBalcony,hasMamad,handicapFriendly,entranceDate,furniture,publishedDays,description,Unnamed: 23
0,פתח תקווה,דירה,5.5,137,רפאלי שרגא,3,אם המושבות החדשה,3600000,6.0,קומה 11 מתוך 19,...,שמור,TRUE,FALSE,TRUE,TRUE,גמיש,לא צויין,7,"למכירה 5.5 חדרים ענקית, מרווחת , מוארת , קומה ...",NaN


In [ ]:
len(df.columns)

24

In [ ]:
len(df)

699

In [ ]:
for column in df.columns:
    print(f"The data type of column '{column}' is: {df[column].dtype}")

The data type of column 'City' is: object
The data type of column 'type' is: object
The data type of column 'room_number' is: object
The data type of column 'Area' is: object
The data type of column 'Street' is: object
The data type of column 'number_in_street' is: object
The data type of column 'city_area' is: object
The data type of column 'price' is: object
The data type of column 'num_of_images' is: float64
The data type of column 'floor_out_of' is: object
The data type of column 'hasElevator ' is: object
The data type of column 'hasParking ' is: object
The data type of column 'hasBars ' is: object
The data type of column 'hasStorage ' is: object
The data type of column 'condition ' is: object
The data type of column 'hasAirCondition ' is: object
The data type of column 'hasBalcony ' is: object
The data type of column 'hasMamad ' is: object
The data type of column 'handicapFriendly ' is: object
The data type of column 'entranceDate ' is: object
The data type of column 'furniture ' 

In [ ]:
# stage_2 - prepare the data
# 2-a
print(f'before -> {len(df)} rows.')
df.dropna(subset=['price'], inplace=True)
print(f'after -> {len(df)} rows.')

before -> 699 rows.
after -> 694 rows.


In [ ]:
# 2-b
df['price'] = df['price'].replace('[^\d.]', '', regex=True)
df['price'] = pd.to_numeric(df['price'], errors='coerce')
df['price'] = df['price'].astype(float)

df['price'].dtype

dtype('float64')

In [ ]:
# 2-c
df['Area'] = df['Area'].replace('[^\d.]', '', regex=True)
df['Area'] = pd.to_numeric(df['Area'], errors='coerce')
df['Area'] = df['Area'].astype(float)

df['Area'].dtype

dtype('float64')

In [ ]:
# 2-d
def remove_punctuation(text):
    if isinstance(text, str):
        return text.translate(str.maketrans('', '', string.punctuation))
    return text

#text = "hel)(lo, 'my-n/am]e![ is. Yaron"
#print(remove_punctuation(text))

df['Street'] = df['Street'].apply(remove_punctuation)
df['Street'].dtype

dtype('O')

In [ ]:
# 2-e
def convert_int(value):
    try:
        return int(value)
    except:
        return 0

df['floor'] = df['floor_out_of'].str.split(' ', expand=True)[1]
df['floor'] = pd.to_numeric(df['floor'].apply(convert_int))

df['floor'].unique()


array([11,  6,  2,  3,  7,  4, 13,  9, 14,  0,  1,  5, 10, 19,  8, 15, 25,
       17, 12, 18, 29, 16, 20])

In [ ]:
df['total_floor'] = df['floor_out_of'].str.split(' ', expand=True)[3]
df['total_floor'] = pd.to_numeric(df['total_floor'].apply(convert_int))
df['total_floor'].unique()

array([19,  9,  7,  6,  8, 24, 14, 16, 21, 28,  3,  0,  4,  2, 10,  5, 18,
       20, 13, 12, 30, 15, 26, 25,  1, 17, 11, 27, 22])

In [ ]:
df['entranceDate '].unique()

array(['גמיש', 'מיידי', '01/07/2023', 'לא צויין', '01/11/2023',
       '01/12/2023', '01/06/2023', '01/01/2023', 'גמיש ', '01/08/2023',
       '01/01/2026', '01/06/2024', '01/05/2024', '01/10/2023',
       '01/12/2024', '01/07/2024', '01/01/2024', '01/02/2024',
       '01/11/2022', '01/09/2024', '01/11/2025', '01/11/2024',
       '01/04/2024', '01/03/2025', '01/02/2025', '01/02/2023',
       '01/03/2024'], dtype=object)

In [ ]:
# Define today's date
today = pd.Timestamp.now().strftime('%d/%m/%Y')
today = pd.to_datetime(today)

# Create a copy of the original DataFrame, as we're going to modify the entranceDate column
df_temp = df.copy()

# Convert to datetime format where possible
df_temp['entranceDate '] = pd.to_datetime(df_temp['entranceDate '], errors='coerce', dayfirst=True)

# Create the entrance_date column and initially set all values to not_defined
df['entrance_date'] = 'not_defined'

# Update the entrance_date column based on conditions

# Condition for 'flexible'
df.loc[df['entranceDate '].str.contains('גמיש'), 'entrance_date'] = 'flexible'

# Condition for 'less_than_6_months'
df.loc[df['entranceDate '].str.contains('מיידי'), 'entrance_date'] = 'less_than_6_months'

# Condition for 'months_6_12'
df.loc[((df_temp['entranceDate '] - today).dt.days / 30 >= 6) & ((df_temp['entranceDate '] - today).dt.days / 30 <= 12), 'entrance_date'] = 'months_6_12'

# Condition for 'above_year'
df.loc[((df_temp['entranceDate '] - today).dt.days / 30 > 12), 'entrance_date'] = 'above_year'



<ipython-input-13-2d55bef3ed1e>:3: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  today = pd.to_datetime(today)


In [ ]:
df['entrance_date'].unique()

array(['flexible', 'less_than_6_months', 'not_defined', 'above_year',
       'months_6_12'], dtype=object)

In [ ]:
df.columns

Index(['City', 'type', 'room_number', 'Area', 'Street', 'number_in_street',
       'city_area', 'price', 'num_of_images', 'floor_out_of', 'hasElevator ',
       'hasParking ', 'hasBars ', 'hasStorage ', 'condition ',
       'hasAirCondition ', 'hasBalcony ', 'hasMamad ', 'handicapFriendly ',
       'entranceDate ', 'furniture ', 'publishedDays ', 'description ',
       'Unnamed: 23', 'floor', 'total_floor', 'entrance_date'],
      dtype='object')

In [ ]:
# 2-g
print('hasElevator ',df['hasElevator '].unique())
print('hasParking ',df['hasParking '].unique())
print('hasBars ',df['hasBars '].unique())
print('hasStorage ',df['hasStorage '].unique())
print('hasAirCondition ',df['hasAirCondition '].unique())
print('hasBalcony ',df['hasBalcony '].unique())
print('hasMamad ',df['hasMamad '].unique())
print('handicapFriendly ',df['handicapFriendly '].unique())

hasElevator  ['TRUE' 'FALSE' '1' '0' 'כן' 'לא' nan 'יש מעלית' 'אין מעלית' 'אין' 'יש'
 'no' 'yes']
hasParking  ['FALSE' 'TRUE' '0' '1' 'כן' 'לא' nan 'יש חנייה' 'יש' 'אין' 'אין חניה'
 'יש חניה' 'no' 'yes']
hasBars  ['TRUE' 'FALSE' '0' '1' 'לא' 'כן' nan 'אין סורגים' 'יש סורגים' 'יש' 'אין'
 'no' 'yes']
hasStorage  ['FALSE' 'TRUE' '1' '0' 'כן' 'לא' nan 'אין מחסן' 'יש מחסן' 'יש' 'אין' 'no'
 'yes']
hasAirCondition  ['TRUE' 'FALSE' '1' '0' 'כן' 'לא' nan 'יש מיזוג אויר' 'יש' 'אין'
 'אין מיזוג אויר' 'יש מיזוג אוויר' 'no' 'yes']
hasBalcony  ['FALSE' 'TRUE' '0' '1' 'לא' 'כן' nan 'אין מרפסת' 'יש מרפסת' 'יש' 'אין'
 'no' 'yes']
hasMamad  ['TRUE' 'FALSE' '1' '0' 'לא' 'כן' 'yes' nan 'אין ממ״ד' 'יש ממ״ד' 'יש'
 'אין' 'יש ממ"ד' 'אין ממ"ד' 'no']
handicapFriendly  ['TRUE' 'FALSE' '0' '1' 'לא' 'כן' nan 'לא נגיש לנכים' 'נגיש לנכים' 'נגיש'
 'לא נגיש']


In [ ]:
def convert_boolean(value):
    if isinstance(value, str):
        if ('אין' in value) or ('לא נגיש') in value:
            return 0
        elif ('יש') in value or ('נגיש') in value:
            return 1
    return 0

columns_to_convert = ['hasElevator ', 'hasParking ', 'hasBars ', 'hasStorage ', 'hasAirCondition ', 'hasBalcony ', 'hasMamad ', 'handicapFriendly ']
df[columns_to_convert] = df[columns_to_convert].applymap(convert_boolean).astype(bool).astype(int)


In [ ]:
# 2-g
print('hasElevator ',df['hasElevator '].unique())
print('hasParking ',df['hasParking '].unique())
print('hasBars ',df['hasBars '].unique())
print('hasStorage ',df['hasStorage '].unique())
print('hasAirCondition ',df['hasAirCondition '].unique())
print('hasBalcony ',df['hasBalcony '].unique())
print('hasMamad ',df['hasMamad '].unique())
print('handicapFriendly ',df['handicapFriendly '].unique())

hasElevator  [0 1]
hasParking  [0 1]
hasBars  [0 1]
hasStorage  [0 1]
hasAirCondition  [0 1]
hasBalcony  [0 1]
hasMamad  [0 1]
handicapFriendly  [0 1]


In [ ]:
# stage_3
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

In [ ]:
# prepare the df
# Convert 'price', 'room_number', 'Area' columns to string
df[['price', 'room_number', 'Area']] = df[['price', 'room_number', 'Area']].astype(str)

# Perform string operations and then convert to numeric
df['price'] = pd.to_numeric(df['price'].str.replace('[^\d.]', ''), errors='coerce')
df['room_number'] = pd.to_numeric(df['room_number'].str.replace('[^\d.]', ''), errors='coerce')
df['Area'] = pd.to_numeric(df['Area'].str.replace('[^\d.]', ''), errors='coerce')

# Convert binary features to 0 and 1
binary_columns = ['hasElevator ', 'hasParking ', 'hasBars ', 'hasStorage ', 'hasAirCondition ', 'hasBalcony ', 'hasMamad ', 'handicapFriendly ']

# Ensure the binary columns are of string type
df[binary_columns] = df[binary_columns].astype(str)

# Now apply the string operation and convert to integer
df[binary_columns] = df[binary_columns].applymap(lambda x: 1 if 'יש' in x else 0)



<ipython-input-20-8a35305510c3>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['price'] = pd.to_numeric(df['price'].str.replace('[^\d.]', ''), errors='coerce')
<ipython-input-20-8a35305510c3>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df['room_number'] = pd.to_numeric(df['room_number'].str.replace('[^\d.]', ''), errors='coerce')
<ipython-input-20-8a35305510c3>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Area'] = pd.to_numeric(df['Area'].str.replace('[^\d.]', ''), errors='coerce')


In [ ]:
# Necessary imports
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import ElasticNetCV
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

# First, we will handle categorical variables using one-hot encoding
df = pd.get_dummies(df, drop_first=True)

# Handle any remaining NaN values
df.fillna(0, inplace=True)

# Select predictor columns.
# Here I've selected all columns except 'price'. You can choose the columns as per your requirement
X = df.drop('price', axis=1)

# Select target column
y = df['price']

# Standardize the predictor columns
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the ElasticNetCV model
model = ElasticNetCV(l1_ratio=[.1, .5, .7, .9, .95, .99, 1], cv=10, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Use the model to make predictions
y_pred = model.predict(X_test)

# Now, we can evaluate the model
print("R^2 Score: ", metrics.r2_score(y_test, y_pred))
print("Mean Absolute Error: ", metrics.mean_absolute_error(y_test, y_pred))
print("Mean Squared Error: ", metrics.mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error: ", np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

# Print the optimal alpha and l1 ratio
print('Optimal alpha: ', model.alpha_)
print('Optimal l1 ratio: ', model.l1_ratio_)


R^2 Score:  0.5387373843325127
Mean Absolute Error:  704104.8613541232
Mean Squared Error:  2038065193863.8083
Root Mean Squared Error:  1427608.2074097951
Optimal alpha:  18573.304307247483
Optimal l1 ratio:  1.0


In [ ]:
# model.py
pickle.dump(model, open("model.pkl","wb"))

In [ ]:
from flask import Flask, render_template, request
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNetCV
from sklearn.preprocessing import StandardScaler
import pickle

app = Flask(__name__)

# Load the trained model and scaler
model = pickle.load(open("model.pkl", "rb"))

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/predict", methods=["POST"])
def predict():
    # Get the input data from the form
    area = float(request.form["area"])
    rooms = int(request.form["rooms"])
    has_elevator = int(request.form["elevator"])
    has_parking = int(request.form["parking"])
    has_bars = int(request.form["bars"])
    has_storage = int(request.form["storage"])
    has_air_condition = int(request.form["air_condition"])
    has_balcony = int(request.form["balcony"])
    has_mamad = int(request.form["mamad"])
    is_handicap_friendly = int(request.form["handicap_friendly"])

    # Create a DataFrame from the input data
    data = {
        "Area": [area],
        "room_number": [rooms],
        "hasElevator": [has_elevator],
        "hasParking": [has_parking],
        "hasBars": [has_bars],
        "hasStorage": [has_storage],
        "hasAirCondition": [has_air_condition],
        "hasBalcony": [has_balcony],
        "hasMamad": [has_mamad],
        "handicapFriendly": [is_handicap_friendly]
    }
    df = pd.DataFrame(data)

    # Convert binary features to 0 and 1
    binary_columns = ['hasElevator', 'hasParking', 'hasBars', 'hasStorage', 'hasAirCondition', 'hasBalcony', 'hasMamad', 'handicapFriendly']
    df[binary_columns] = df[binary_columns].astype(str)
    df[binary_columns] = df[binary_columns].applymap(lambda x: 1 if 'is' in x else 0)

    # Standardize the input data using the scaler
    scaler = StandardScaler()
    df_scaled = scaler.fit_transform(df)

    # Predict the apartment price
    price = model.predict(df_scaled)[0]

    return render_template("index.html", price=price)

if __name__ == "__main__":
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat


In [ ]:
# flask.api

app = Flask(__name__)
rf_model = pickle.load(open('model.pkl', 'rb'))

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict',methods=['POST'])
def predict():
    '''
    For rendering results on HTML GUI
    '''
    features = request.form.getlist('feature')

    final_features = [features]
    prediction = rf_model.predict(final_features)[0]
    #risk = rf_model.predict_proba(final_features)[0][1]

    prediction_text = f'The price is: {prediction}'

    return render_template('index.html', prediction_text='{}'.format(output_text))


if __name__ == "__main__":
    port = int(os.environ.get('PORT', 5000))

    app.run(host='0.0.0.0', port=port,debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
